In [1]:
from LSTM_Flare import LSTM_Flare
from sklearn.utils import class_weight
from keras.models import *
import numpy as np
import csv


Using TensorFlow backend.


In [2]:
flare_label = 'C'
train_again = False
filepath = './data/LSTM_{flare_label}_sample_run/'.format(flare_label=flare_label)
n_features = 0
if flare_label == 'C':
    n_features = 14
elif flare_label == 'M':
    n_features = 22
elif flare_label == 'M5':
    n_features = 20
start_feature = 5
mask_value = 0
series_len = 10
epochs = 7
batch_size = 256
nclass = 2
result_file = './results/LSTM_{flare_label}_sample_run/output.csv'.format(flare_label=flare_label)

In [3]:
lstm_flare = LSTM_Flare()

In [4]:
if train_again:
    # Train
    X_train_data, y_train_data = lstm_flare.load_data(datafile=filepath + 'normalized_training.csv',
                                           flare_label=flare_label, series_len=series_len,
                                           start_feature=start_feature, n_features=n_features,
                                           mask_value=mask_value)

    X_train = np.array(X_train_data)
    y_train = np.array(y_train_data)
    y_train_tr = lstm_flare.data_transform(y_train)

    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(y_train), y_train)
    class_weight_ = {0: class_weights[0], 1: class_weights[1]}
    # print(class_weight_)

    model = lstm_flare.lstm(nclass, n_features, series_len)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train_tr,
                        epochs=epochs, batch_size=batch_size,
                        verbose=False, shuffle=True, class_weight=class_weight_)
    model.save('./pretrained_model/LSTM_{flare_label}_sample_run/model.h5'.format(flare_label=flare_label))
else:
    model = load_model('./pretrained_model/LSTM_{flare_label}_sample_run/model.h5'.format(flare_label=flare_label))

In [5]:
# Test
X_test_data, y_test_data = lstm_flare.load_data(datafile=filepath + 'normalized_testing.csv',
                                     flare_label=flare_label, series_len=series_len,
                                     start_feature=start_feature, n_features=n_features,
                                     mask_value=mask_value)
X_test = np.array(X_test_data)
y_test = np.array(y_test_data)
y_test_tr = lstm_flare.data_transform(y_test)

classes = model.predict(X_test, batch_size=batch_size, verbose=0, steps=None)

with open(result_file, 'w', encoding='UTF-8') as result_csv:
    w = csv.writer(result_csv)
    with open(filepath + 'normalized_testing.csv', encoding='UTF-8') as data_csv:
        reader = csv.reader(data_csv)
        i = -1
        for line in reader:
            if i == -1:
                line.insert(0, 'Predicted Label')
            else:
                if (classes[i][0] >= 0.5 and flare_label == 'C') or (classes[i][0] >= 0.75 and flare_label == 'M5') or (classes[i][0] >= 0.6 and flare_label == 'M'):
                    line.insert(0, 'Positive')
                else:
                    line.insert(0, 'Negative')
            i += 1
            w.writerow(line)

(185, 10, 14)
